In [14]:
import environment.job_search as job_search_env
import argparse
import os
import random

import ray
from ray import tune
from ray.tune.registry import register_env

In [23]:
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.algorithms.ppo import PPOConfig

AttributeError: module 'ray._private' has no attribute 'worker'

In [20]:
from ray import tune
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.tune.registry import register_env
from torch import nn

In [31]:
def env_creator(args):
    env = job_search_env.env()
    return env

In [21]:
register_env("JobSearch", lambda config: PettingZooEnv(env_creator(config)))

In [22]:
ray.init()

AssertionError: 